In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import timedelta
import pandas.api.types as ptypes
import sys
from pandas.tseries.offsets import *

# parameters for animated plot: param_start_date= start date of the plot animation, end date is 4 month after
param_start_date='05/20/2018'

In [2]:
#read tweets in to the dataframe

name_tweets = "results.json"

with open(name_tweets, "r") as file:
    tweets_df = pd.read_json(file, orient='index')
print("Number of tweets: ", len(tweets_df))
print(tweets_df.columns)

#sorting and re-indexing
tweets_df.sort_index(inplace=True)
tweets_df = tweets_df.reset_index(drop=True)

print("Oldest tweet: ", tweets_df["timestamp"][0])
print("Latest tweet", tweets_df["timestamp"][len(tweets_df)-1])

Number of tweets:  2348
Index(['predicted changes', 'text', 'timestamp', 'tweet', 'value change'], dtype='object')
Oldest tweet:  2015-06-14 21:18:23
Latest tweet 2018-09-25 16:45:36


In [3]:
#modifying data
tweets_df['difference'] = np.where(tweets_df["value change"]==tweets_df["predicted changes"], 1, 0)
print("Difference=1 when predicted change is equal to the real change")
tweets_df.head()

Difference=1 when predicted change is equal to the real change


,predicted changes,text,timestamp,tweet,value change,difference
0,SMANEG,tuesday will be a big day for our country to d...,2015-06-14 21:18:23,Tuesday will be a big day for our country to d...,MEDNEG,0
1,MEDNEG,https https i d vote twice for you if i could ...,2015-06-15 11:30:53,"""@FuturisticHub: @realDonaldTrump I'd vote twi...",MEDPOS,0
2,MEDNEG,https https trump time for a true leader to le...,2015-06-15 11:41:38,"""@fyrftr211: @realDonaldTrump #Trump2016. Time...",MEDPOS,0
3,SMAPOS,https politicians are not going to get us out ...,2015-06-15 11:43:49,"""@LooneyTunes002: Politicians are not going to...",MEDPOS,0
4,MEDPOS,live on periscope https,2015-06-15 14:55:16,LIVE on #Periscope https://t.co/H8lRtZBGEb,MEDPOS,1


In [12]:
#read stock market data in to the dataframe

with open('DJI-20years-daily-close.json', 'r') as myfile:
    data1=myfile.read()

obj = json.loads(data1)
orig_stock_df = pd.DataFrame.from_dict(obj, orient='index')

print(len(orig_stock_df))
print("The oldest stock market date: ", orig_stock_df["4. close"].index[0])
print("The latest stock market date: ", orig_stock_df["4. close"].index[len(orig_stock_df["4. close"])-1])
print(orig_stock_df)

4725
The oldest stock market date:  2000-01-03
The latest stock market date:  2018-10-11
              4. close
2000-01-03  11357.5098
2000-01-04  10997.9297
2000-01-05  11122.6504
2000-01-06  11253.2598
2000-01-07  11522.5596
2000-01-10  11572.2002
2000-01-11  11511.0801
2000-01-12  11551.0996
2000-01-13  11582.4297
2000-01-14  11722.9805
2000-01-18  11560.7197
2000-01-19  11489.3604
2000-01-20  11351.2998
2000-01-21  11251.7100
2000-01-24  11008.1699
2000-01-25  11029.8896
2000-01-26  11032.9902
2000-01-27  11028.0195
2000-01-28  10738.8701
2000-01-31  10940.5303
2000-02-01  11041.0498
2000-02-02  11003.2002
2000-02-03  11013.4404
2000-02-04  10963.7998
2000-02-07  10905.7900
2000-02-08  10957.5996
2000-02-09  10699.1602
2000-02-10  10643.6299
2000-02-11  10425.2100
2000-02-14  10519.8398
...                ...
2018-08-30  25986.9199
2018-08-31  25964.8203
2018-09-04  25952.4805
2018-09-05  25974.9902
2018-09-06  25995.8691
2018-09-07  25916.5391
2018-09-10  25857.0703
2018-09-11  25

In [38]:
#counting the daily change out of the stock market data

sample_size = orig_stock_df.shape[0]-1
endpoints = orig_stock_df[1:]
startpoints = orig_stock_df[:sample_size]

#print (startpoints.index)
#print (endpoints.index)

#save the startpoint index to be used as the date. This way yesterday's bin will hold value
#of change between yesterdays closing and todays closing, so yesterdays tweets get the value impact of today's
#stock market value development
indexes = startpoints.index

#let's reset the indexes
startpoints = startpoints.reset_index()
endpoints = endpoints.reset_index()
#print (startpoints.index)
#print (endpoints.index)

daily_change = endpoints["4. close"] - startpoints["4. close"]
change = []
for i in range(len(daily_change)):
    change.append(daily_change[i])

change.append(0)

orig_stock_df['daily change']=pd.Series(change, index=orig_stock_df.index)
print(orig_stock_df.head())

              4. close  daily change
2000-01-03  11357.5098     -359.5801
2000-01-04  10997.9297      124.7207
2000-01-05  11122.6504      130.6094
2000-01-06  11253.2598      269.2998
2000-01-07  11522.5596       49.6406


In [39]:
#combine the tweets and stock market data
classified = pd.DataFrame(columns=["date", "timestamp", "text", "tweet", "predicted changes", "orig changes", "difference", 
                                   "orig stock", "prev stock", "daily change"], 
                          index=range(orig_stock_df.shape[0]*3))
print("classified shape:", classified.shape, "\norig_stock_df shape:", orig_stock_df.shape)
time_bins = {}
latest_date_ind = 0

#Add date to the tweets data
tweets_df['date'] = tweets_df["timestamp"].dt.date
maksimi=0
j=0

for i, t in enumerate(orig_stock_df.index):    

    stock_date=datetime.strptime(t, '%Y-%m-%d').date()

    if stock_date in tweets_df['date'].values: 
        tweets=tweets_df.loc[tweets_df["date"]==stock_date]
        tweets.sort_index(inplace=True)
        tweets = tweets.reset_index(drop=True)
        for j in range(len(tweets)):

            classified["date"][maksimi] = t
            classified["timestamp"][maksimi] = tweets["timestamp"][j]
            classified["text"][maksimi] = tweets["text"][j]
            classified["tweet"][maksimi] = tweets["tweet"][j]
            classified["predicted changes"][maksimi] = tweets["predicted changes"][j]
            classified["orig changes"][maksimi] = tweets["value change"][j]
            classified["difference"][maksimi] = tweets["difference"][j]
            classified["orig stock"][maksimi] = orig_stock_df["4. close"][i] 
            #previous stock market value for animated plot
            classified["prev stock"][maksimi] = orig_stock_df["4. close"][i-1]
            classified["daily change"][maksimi] = orig_stock_df["daily change"][i]
            maksimi=maksimi+1
    else:
        classified["date"][maksimi] = t
        classified["timestamp"][maksimi] = datetime.strptime(t, '%Y-%m-%d')
        classified["orig stock"][maksimi] = orig_stock_df["4. close"][i] 
        classified["prev stock"][maksimi] = orig_stock_df["4. close"][i-1] 
        classified["daily change"][maksimi] = orig_stock_df["daily change"][i]
        maksimi=maksimi+1

classified.head()

classified shape: (14175, 10) 
orig_stock_df shape: (4725, 2)


,date,timestamp,text,tweet,predicted changes,orig changes,difference,orig stock,prev stock,daily change
0,2000-01-03,2000-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN,11357.5,25052.8,-359.58
1,2000-01-04,2000-01-04 00:00:00,NaN,NaN,NaN,NaN,NaN,10997.9,11357.5,124.721
2,2000-01-05,2000-01-05 00:00:00,NaN,NaN,NaN,NaN,NaN,11122.7,10997.9,130.609
3,2000-01-06,2000-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,11253.3,11122.7,269.3
4,2000-01-07,2000-01-07 00:00:00,NaN,NaN,NaN,NaN,NaN,11522.6,11253.3,49.6406


In [40]:
classified_drop=classified[classified["timestamp"].notnull()]
print("The oldest stock market date: ", classified_drop["timestamp"][0])
print("The latest stock market date: ", classified_drop["timestamp"][len(classified_drop["timestamp"])-1])

The oldest stock market date:  2000-01-03 00:00:00
The latest stock market date:  2018-10-11 00:00:00


In [41]:
#Business day offset
isBusinessDay = BDay().onOffset
match_series = pd.to_datetime(classified_drop['date']).map(isBusinessDay)
classified_drop[match_series]

,date,timestamp,text,tweet,predicted changes,orig changes,difference,orig stock,prev stock,daily change
0,2000-01-03,2000-01-03 00:00:00,NaN,NaN,NaN,NaN,NaN,11357.5,25052.8,-359.58
1,2000-01-04,2000-01-04 00:00:00,NaN,NaN,NaN,NaN,NaN,10997.9,11357.5,124.721
2,2000-01-05,2000-01-05 00:00:00,NaN,NaN,NaN,NaN,NaN,11122.7,10997.9,130.609
3,2000-01-06,2000-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,11253.3,11122.7,269.3
4,2000-01-07,2000-01-07 00:00:00,NaN,NaN,NaN,NaN,NaN,11522.6,11253.3,49.6406
5,2000-01-10,2000-01-10 00:00:00,NaN,NaN,NaN,NaN,NaN,11572.2,11522.6,-61.1201
6,2000-01-11,2000-01-11 00:00:00,NaN,NaN,NaN,NaN,NaN,11511.1,11572.2,40.0195
7,2000-01-12,2000-01-12 00:00:00,NaN,NaN,NaN,NaN,NaN,11551.1,11511.1,31.3301
8,2000-01-13,2000-01-13 00:00:00,NaN,NaN,NaN,NaN,NaN,11582.4,11551.1,140.551
9,2000-01-14,2000-01-14 00:00:00,NaN,NaN,NaN,NaN,NaN,11723,11582.4,-162.261


In [42]:
#write data in to the file for descriptive statistics
with open('tweet_stock_results.json', 'w') as file:
    file.write(classified_drop.to_json(orient='index'))

In [43]:
#select the data based on the param_start_date
classified_drop=classified_drop.loc[classified_drop["timestamp"] > datetime.strptime(param_start_date, '%m/%d/%Y')]

print(classified_drop["timestamp"].values[0])
#print(len(tweets_df))
print(classified_drop["timestamp"].values[len(classified_drop)-1])

#drop duplicates for animated plot
classified_drop.drop_duplicates(["date"],inplace=True)

#taking the text column if the predicted value is equal to value change
classified_drop["text"] = np.where(classified_drop["difference"]==1, classified_drop["text"], "")

#replacing NaN values as empty  
classified_drop["text"] = np.where(classified_drop["text"]==np.nan, "", classified_drop["text"])
classified_drop["predicted changes"] = np.where(classified_drop["predicted changes"]==np.nan, "", classified_drop["predicted changes"])

classified_drop.head()

2018-05-21 11:27:42
2018-10-11 00:00:00


,date,timestamp,text,tweet,predicted changes,orig changes,difference,orig stock,prev stock,daily change
5551,2018-05-21,2018-05-21 11:27:42,,China has agreed to buy massive amounts of ADD...,BIGPOS,BIGNEG,0,25013.3,24715.1,-178.879
5553,2018-05-22,2018-05-22 00:00:00,,NaN,NaN,NaN,NaN,24834.4,25013.3,52.4003
5554,2018-05-23,2018-05-23 01:13:46,,If the person placed very early into my campai...,SMAPOS,MEDNEG,0,24886.8,24834.4,-75.0507
5556,2018-05-24,2018-05-24 12:34:29,,"Not surprisingly, the GREAT Men &amp; Women of...",BIGNEG,MEDNEG,0,24811.8,24886.8,-58.67
5558,2018-05-25,2018-05-25 23:13:19,,...but complain and obstruct. They made only b...,MEDPOS,BIGNEG,0,24753.1,24811.8,-391.641


In [44]:
del classified_drop["date"]
del classified_drop["tweet"]
del classified_drop["orig changes"]
del classified_drop["difference"]
del classified_drop["orig stock"]
del classified_drop["daily change"]
np.savetxt("animation.txt", classified_drop.values, encoding='utf-8', fmt='%s', delimiter=',')